In [19]:
import pandas as pd
import joblib
import numpy as np
import pandas as pd
import geopandas as gpd
from geopy.geocoders import Nominatim
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [20]:
dataset=pd.read_csv('Crime_FIR_Dataset.csv')
data=pd.read_csv('Crime_FIR_Dataset.csv')
data

,Timestamp,City Name,Latitude,Longitude,FIR No.,Crime Type,Date Crime,Time Crime,Gender,Age,SafetyIndex
0,2019/12/10 12:52:02 PM GMT+5:30,Andheri,19.115154,72.866151,850,Murder,23-11-2019,15:00,Male,72,2.0
1,2019/12/10 12:56:40 PM GMT+5:30,Andheri,19.119719,72.847316,839,Theft,19-11-2019,18:30,Male,39,2.0
2,2019/12/10 12:59:08 PM GMT+5:30,Andheri,19.119415,72.847059,838,Theft,19-11-2019,16:49,Male,57,2.0
3,2019/12/10 1:03:00 PM GMT+5:30,Santacruz,19.082584,72.838128,575,Theft,20-11-2019,20:00,Female,30,2.7
4,2019/12/10 1:04:19 PM GMT+5:30,Andheri,19.116373,72.853582,835,Murder,18-11-2019,11:21,Male,35,2.0
...,...,...,...,...,...,...,...,...,...,...,...
208,2019/12/13 8:19:38 PM GMT+5:30,RCF,19.043009,72.895914,431,Robbery,19-10-2019,11:30,Male,32,3.5
209,2019/12/13 8:21:08 PM GMT+5:30,RCF,19.041386,72.908102,425,Theft,16-10-2019,11:41,Male,35,3.0
210,2019/12/13 8:22:30 PM GMT+5:30,RCF,19.040413,72.902781,425,Theft,16-10-2019,08:30,Male,35,3.0
211,2019/12/13 8:24:10 PM GMT+5:30,RCF,19.036261,72.890221,410,Robbery,08-10-2019,17:35,Female,23,3.0


In [21]:
data['Date-Time'] = data['Date Crime'] + " " + data['Time Crime']
data['Date-Time'] = pd.to_datetime(data['Date-Time'],  errors='coerce')

In [22]:
Column = data['Date-Time']

db=pd.DataFrame({
    "year": Column.dt.year, 
    "month": Column.dt.month, 
    "day": Column.dt.day, 
    "hour": Column.dt.hour, 
    "minute": Column.dt.minute, 
    "weekday": Column.dt.weekday
})

db['Robbery'] = data.apply(lambda x: 1 if x["Crime Type"]=="Robbery" else 0, axis=1)
db['Theft'] = data.apply(lambda x: 1 if x["Crime Type"]=="Theft" else 0, axis=1)
db['Murder'] = data.apply(lambda x: 1 if x["Crime Type"]=="Murder" else 0, axis=1)
db

,year,month,day,hour,minute,weekday,Robbery,Theft,Murder
0,2019,11,23,15,0,5,0,0,1
1,2019,11,19,18,30,1,0,1,0
2,2019,11,19,16,49,1,0,1,0
3,2019,11,20,20,0,2,0,1,0
4,2019,11,18,11,21,0,0,0,1
...,...,...,...,...,...,...,...,...,...
208,2019,10,19,11,30,5,1,0,0
209,2019,10,16,11,41,2,0,1,0
210,2019,10,16,8,30,2,0,1,0
211,2019,8,10,17,35,5,1,0,0


In [23]:
dataset1=dataset.drop('Timestamp',axis=1)
data1=pd.concat([db,dataset1["Latitude"],dataset1["Longitude"],dataset1["Crime Type"],dataset1["SafetyIndex"]],axis=1)
data1.dropna(inplace=True)
data1

,year,month,day,hour,minute,weekday,Robbery,Theft,Murder,Latitude,Longitude,Crime Type,SafetyIndex
0,2019,11,23,15,0,5,0,0,1,19.115154,72.866151,Murder,2.0
1,2019,11,19,18,30,1,0,1,0,19.119719,72.847316,Theft,2.0
2,2019,11,19,16,49,1,0,1,0,19.119415,72.847059,Theft,2.0
3,2019,11,20,20,0,2,0,1,0,19.082584,72.838128,Theft,2.7
4,2019,11,18,11,21,0,0,0,1,19.116373,72.853582,Murder,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,2019,10,19,11,30,5,1,0,0,19.043009,72.895914,Robbery,3.5
209,2019,10,16,11,41,2,0,1,0,19.041386,72.908102,Theft,3.0
210,2019,10,16,8,30,2,0,1,0,19.040413,72.902781,Theft,3.0
211,2019,8,10,17,35,5,1,0,0,19.036261,72.890221,Robbery,3.0


In [31]:
X=data1.iloc[:,[1,2,3,4,5,9,10]].values
Y_classifier=data1.iloc[:,[6,7,8]].values
X_train, X_test, Y_train, Y_test = train_test_split(X, Y_classifier, test_size=0.25, random_state=50)

RFClassifier = RandomForestClassifier(n_estimators=300)
RFClassifier.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [32]:
Y_regressor=data1.iloc[:,[12]].values
XR_train, XR_test, YR_train, YR_test = train_test_split(X, Y_regressor, test_size=0.25, random_state=50)

RFRegressor = RandomForestRegressor(n_estimators=300)
RFRegressor.fit(XR_train, YR_train.ravel())

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=300,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [33]:
locator = Nominatim(user_agent="myGeocoder", format_string="%s, Mumbai, India")
location = locator.geocode("Kurla")

In [34]:
lat=[location.latitude]
log=[location.longitude]
latlong=pd.DataFrame({'latitude':lat,'longitude':log})

DT= "2019-03-01 22:21"
latlong['Date-Time']=DT
test_data=latlong
cols = test_data.columns.tolist()
cols = cols[-1:] + cols[:-1]
test_data = test_data[cols]

test_data['Date-Time'] = pd.to_datetime(test_data['Date-Time'].astype(str), errors='coerce')
test_data['Date-Time'] = pd.to_datetime(test_data['Date-Time'], format = '%d/%m/%Y %H:%M:%S')
test_data

,Date-Time,latitude,longitude
0,2019-03-01 22:21:00,19.06528,72.87938


In [37]:
TestColumn = test_data.iloc[:,0]

DT=pd.DataFrame({
    "year": TestColumn.dt.year, 
    "month": TestColumn.dt.month, 
    "day": TestColumn.dt.day, 
    "hour": TestColumn.dt.hour, 
    "minute": TestColumn.dt.minute, 
    "weekday": TestColumn.dt.weekday
})
print(DT)

   year  month  day  hour  minute  weekday
0  2019      3    1    22      21        4


In [38]:
test_data=test_data.drop('Date-Time',axis=1)

final=pd.concat([DT,test_data],axis=1)
print(final)

   year  month  day  hour  minute  weekday  latitude  longitude
0  2019      3    1    22      21        4  19.06528   72.87938


In [39]:
X_test=final.iloc[:,[1,2,3,4,5,6,7]].values
print(X_test)

[[ 3.         1.        22.        21.         4.        19.0652797
  72.8793805]]


In [40]:
CrimeType_Test = RFClassifier.predict(X_test)
SafetyIndex_Test = RFRegressor.predict(X_test)
print(CrimeType_Test, SafetyIndex_Test)

[[0 1 0]] [2.62466667]
